In [2]:
from bs4 import BeautifulSoup
import requests
import csv
import json
import re
import random
import pandas as pd
import os
import datetime
import numpy as np
import time


In [ ]:
# 获取所有站基本信息、站名、站id...选取万马的站

In [ ]:
_headers = {}
_headers.update({"Accept": "*/*",
                "Accept-Encoding": "gzip, deflate, br",
                "Accept-Language": "zh-Hans-US;q=1, en-US;q=0.9, ja-US;q=0.8",
                "User-Agent": "eichong/5.4.5 (iPhone; iOS 14.2; Scale/3.00)",
                "Content-Length": "177",
                "Content-Type": "application/x-www-form-urlencoded",
                "X-Ca-eichong1": "AkTSONKgrrFwmeO1jQ==",
                "X-Ca-eichong2": "dkfbQ9LUoM0B7+mwgNNfNlLTm4zhz5bQNDnTcO5DFYiLelhRQx8342o26UOQtQSOoufP",
                "X-WANMA-Version": "5.4.2",
                "Connection": "keep-alive",
                "Host": "api.eichong.com"})
_body = {"cpyId":"1000",
         "distanceLatitude":"30.278711",
         "distanceLongitude":"120.049588",
         "keyWord":"万马",
         "latitude":"30.29428",
         "longitude":"120.089143",
         "pageNum":"20",
         "pageNumber":"1",
         "range":"",
         "sortType":"distance",
         "userId":"2294868"}
url = "https://api.eichong.com/api/app/powerStation/search.do"
Response = requests.get(url, headers = _headers, data = _body)
pd1 = pd.DataFrame(json.loads(Response.text)["data"])
Wanma_id = pd1[pd1["operateType"]==1]["id"]
# 这里operateType自营就是万马站

In [ ]:
# 即万马站id去获得站的当前详情（主要是minRateId 因为用上面那个获得的minRateId去获得价格详情不准）

In [ ]:
def use_PSID_get_miniRateId(_psid):
    _headers = {}
    _headers.update({"Accept": "*/*",
                    "Accept-Encoding": "gzip, deflate, br",
                    "Accept-Language": "zh-Hans-US;q=1, en-US;q=0.9, ja-US;q=0.8",
                    "User-Agent": "eichong/5.4.5 (iPhone; iOS 14.2; Scale/3.00)",
                    "Content-Length": "74",
                    "Content-Type": "application/x-www-form-urlencoded",
                    "X-Ca-eichong1": "AkTSONCjpLpymeG0hA==",
                    "X-Ca-eichong2": "dTbUMtfTr7pzneS1gKJfTVPU54mVz+elTjumcZ5AEo2CfF1TSRA342o260CavgaOoObG",
                    "X-WANMA-Version": "5.4.2",
                    "Connection": "keep-alive",
                    "Host": "api.eichong.com"})
    _body = {"latitude":"30.264332",
             "longitude":"120.039246",
             "powerStationId":"%s" %(_psid),
             "userId":"2294858"}
    url = "https://api.eichong.com/api/app/powerStation.do"
    Response = requests.post(url, headers = _headers, data = _body)
    return Response

pd_minRateId = pd.DataFrame([], columns=["cityName", "address", "name", "minRateId"])

for i in Wanma_id:
    time.sleep(3+random.random()*4)
    R = use_PSID_get_miniRateId(i)
    cn_t = json.loads(R.text)["data"]["cityName"]
    ad_t = json.loads(R.text)["data"]["address"]
    na_t = json.loads(R.text)["data"]["name"]
    mi_t = json.loads(R.text)["data"]["minRateId"]
    pdt = pd.DataFrame([[cn_t,ad_t,na_t,mi_t]], columns=["cityName", "address", "name", "minRateId"])
    pd_minRateId = pd_minRateId.append(pdt)
    
pd_minRateId.reset_index(drop=True,inplace=True)
pd_minRateId.to_csv("./Wanma_true_minRateId.csv", index = False, encoding="utf_8_sig")

In [ ]:
# 用站的minRateId去获取站的价格详情

In [ ]:
pdresult = pd.DataFrame([], columns=["name", "electricityFee", "serviceFee", "timePeriod", "totalFee"])
Errorlist = [] # 有的minRateId找不到东西 只有一个有这样的问题
for i in pd_minRateId.index:
    time.sleep(3+random.random()*4)
    _name = pd_minRateId.loc[i, "name"]
    _id = pd_minRateId.loc[i, "minRateId"]
    _headers = {}
    _headers.update({"Accept": "*/*",
                    "Accept-Encoding": "gzip, deflate, br",
                    "Accept-Language": "zh-Hans-US;q=1, en-US;q=0.9, ja-US;q=0.8",
                    "User-Agent": "eichong/5.4.5 (iPhone; iOS 14.2; Scale/3.00)",
                    "Content-Length": "36",
                    "Content-Type": "application/x-www-form-urlencoded",
                    "X-Ca-eichong1": "AkTSONOgpLpynuW0hg==",
                    "X-Ca-eichong2": "AkClN6eurs9w65bGgNVbTFKk7IuSvefRQjnRB+xBFo+KfF9WQBQ342o26EOavgaJpObE",
                    "X-WANMA-Version": "5.4.2",
                    "Connection": "keep-alive",
                    "Host": "api.eichong.com"})
    _body = {"id":"%s" %(_id), #16739x1000 #16738x1000
             "orgNo":"1000",
             "rateTypeSource":"0"}

    feeurl = "https://api.eichong.com/api/app/rateinformation/selectRateInfo.do"

    Response2 = requests.post(feeurl, headers = _headers, data = _body)
    if json.loads(Response2.text)['status']==100:
        pdt = pd.DataFrame(json.loads(Response2.text)['data'])
        pdt["name"] = _name
        pdresult = pdresult.append(pdt, ignore_index=True)
    else:
        Errorlist.append(i)

pd3 = pd.merge(pdresult, pd_minRateId, how='left', on="name", left_on=None, right_on=None,
      left_index=False, right_index=False, sort=True,
      suffixes=('_x', '_y'), copy=True, indicator=False)
pd3.to_csv("./WanmaFeeEncrypt_v2.csv", index = False, encoding="utf_8_sig")

In [ ]:
# 在以上两个获得站详情时会出现加密与价格对应出现的情况，尽可能多得获取所有这种映射关系
# 这个映射的字典数字和app上显示的不一样 以==结尾的 app上数值结尾少两位（小数点后面只有2位） 以km结尾的 app上数值尾部是00

In [ ]:
# 半小时爬一次全站
def save_dic(_time):
    _headers = {}
    _headers.update({"Accept": "*/*",
                    "Accept-Encoding": "gzip, deflate, br",
                    "Accept-Language": "zh-Hans-US;q=1, en-US;q=0.9, ja-US;q=0.8",
                    "User-Agent": "eichong/5.4.5 (iPhone; iOS 14.2; Scale/3.00)",
                    "Content-Length": "177",
                    "Content-Type": "application/x-www-form-urlencoded",
                    "X-Ca-eichong1": "AkTSONKgrrFwmeO1jQ==",
                    "X-Ca-eichong2": "dkfbQ9LUoM0B7+mwgNNfNlLTm4zhz5bQNDnTcO5DFYiLelhRQx8342o26UOQtQSOoufP",
                    "X-WANMA-Version": "5.4.2",
                    "Connection": "keep-alive",
                    "Host": "api.eichong.com"})
    _body = {"cpyId":"1000",
             "distanceLatitude":"35.278711",
             "distanceLongitude":"127.049588",
             "keyWord":"万马",
             "latitude":"35.26428",
             "longitude":"127.039143",
             "pageNum":"20",
             "pageNumber":"1",
             "range":"",
             "sortType":"distance",
             "userId":"2294868"}
    url = "https://api.eichong.com/api/app/powerStation/search.do"
    Response = requests.get(url, headers = _headers, data = _body)

    encrypt_fee_dic = {}
    for i in json.loads(Response.text)["data"]:
        encrypt_fee_dic[i['rC4EncryptMaxChargingFee']] = i["maxChargingFee"]
        encrypt_fee_dic[i['rC4EncryptMaxServiceFee']] = i["maxServiceFee"]
        encrypt_fee_dic[i['rC4EncryptMaxCurrentRate']] = i["maxCurrentRate"]
        encrypt_fee_dic[i['rC4EncryptMinChargingFee']] = i["minChargingFee"]
        encrypt_fee_dic[i['rC4EncryptMinServiceFee']] = i["minServiceFee"]
        encrypt_fee_dic[i['rC4EncryptMinCurrentRate']] = i["minCurrentRate"]


    b = json.dumps(encrypt_fee_dic)
    f2 = open('encrypt_fee_dic%s.json' %(_time), 'w')
    f2.write(b)
    f2.close()

hour_check = 11
while(True):
    _now = datetime.datetime.now().__format__('%Y-%m-%d-%H-%M-%S')
    _hour = datetime.datetime.now().hour
    if hour_check!=_hour:
        state = [True, True]
        hour_check=_hour
    if datetime.datetime.now().minute==0 and state[0]:
        save_dic(_now)
        state[0] = False
    elif datetime.datetime.now().minute==30 and state[1]:
        save_dic(_now)
        state[1] = False
        
# 或者用这个 两个的区别就是一个爬全站当前 一个爬自营
dic_by_minRateId = {}
for i in Wanma_id:
    time.sleep(3+random.random()*4)
    R = use_PSID_get_miniRateId(i)
    data = json.loads(R.text)["data"]
    dic_temp = {}
    dic_temp[data['rC4EncryptMaxChargingFee']] = data["maxChargingFee"]
    dic_temp[data['rC4EncryptMaxServiceFee']] = data["maxServiceFee"]
    dic_temp[data['rC4EncryptMaxCurrentRate']] = data["maxCurrentRate"]
    dic_temp[data['rC4EncryptMinChargingFee']] = data["minChargingFee"]
    dic_temp[data['rC4EncryptMinServiceFee']] = data["minServiceFee"]
    dic_temp[data['rC4EncryptMinCurrentRate']] = data["minCurrentRate"]
    dic_by_minRateId.update(dic_temp)
    
all_Encrypt_dic.update(dic_by_minRateId)
_time = datetime.datetime.now().__format__('%Y-%m-%d-%H-%M-%S')
b = json.dumps(all_Encrypt_dic)
f2 = open('encrypt_fee_all_dic%s.json' %(_time), 'w')
f2.write(b)
f2.close()

In [ ]:
# 获得很多映射关系后 每次都存下来 用这个从文件夹里拿出来一次性更新
files= os.listdir("C:/Users/JIE")
Encrypt_dic_list = []
for file in files:
    if file[:7]=="encrypt":
        Encrypt_dic_list.append(file)

all_Encrypt_dic = {}
for i in Encrypt_dic_list:
    f = open("C:/Users/JIE/%s" %(i), 'r')
    content = f.read()
    dic_temp = json.loads(content)
    f.close()
    all_Encrypt_dic.update(dic_temp)

In [ ]:
# 使用映射关系对价格详情进行解密，找出电费服务费总价三个中两个已知的去推出另一个映射关系，更新映射直到无法更新
# 剩下二十个不到，手动更新映射
f = open("encrypt_fee_all_dic2021-04-24-23-22-21.json", "r")
content = f.read()
encrypt_fee_all_dic = json.loads(content)
f.close()

pd_fee = pd.read_csv("./WanmaFeeEncrypt_v2.csv")

def solve(pd_fee, encrypt_fee_all_dic)
    for i in pd_fee.index:
        if pd_fee.loc[i, "electricityFee"] in encrypt_fee_all_dic.keys():
            pd_fee.loc[i, "electricityFee"] = encrypt_fee_all_dic[pd_fee.loc[i, "electricityFee"]]
        if pd_fee.loc[i, "serviceFee"] in encrypt_fee_all_dic.keys():
            pd_fee.loc[i, "serviceFee"] = encrypt_fee_all_dic[pd_fee.loc[i, "serviceFee"]]
        if pd_fee.loc[i, "totalFee"] in encrypt_fee_all_dic.keys():
            pd_fee.loc[i, "totalFee"] = encrypt_fee_all_dic[pd_fee.loc[i, "totalFee"]]
    return pd_fee
        
def get_new_encrypt_fee_dic(pd_fee):
    new_encrypt_fee_dic = {} # 电费服务费总价三者知道其二又可以获得映射
    for i in pd_fee.index:
        if type(pd_fee.loc[i,"electricityFee"])==float and type(pd_fee.loc[i,"serviceFee"])==float and type(pd_fee.loc[i,"totalFee"])==str:
            new_encrypt_fee_dic[pd_fee.loc[i,"totalFee"]] = pd_fee.loc[i,"electricityFee"] + pd_fee.loc[i,"serviceFee"]
        elif type(pd_fee.loc[i,"electricityFee"])==str and type(pd_fee.loc[i,"serviceFee"])==float and type(pd_fee.loc[i,"totalFee"])==float:
            new_encrypt_fee_dic[pd_fee.loc[i,"electricityFee"]] = pd_fee.loc[i,"totalFee"] - pd_fee.loc[i,"serviceFee"]
        elif type(pd_fee.loc[i,"electricityFee"])==float and type(pd_fee.loc[i,"serviceFee"])==str and type(pd_fee.loc[i,"totalFee"])==float:
            new_encrypt_fee_dic[pd_fee.loc[i,"serviceFee"]] = pd_fee.loc[i,"totalFee"] - pd_fee.loc[i,"electricityFee"]
    return new_encrypt_fee_dic

pd_fee = solve(pd_fee, encrypt_fee_all_dic)
new_encrypt_fee_dic = get_new_encrypt_fee_dic(pd_fee)
encrypt_fee_all_dic.update(new_encrypt_fee_dic)

pd_fee = solve(pd_fee, encrypt_fee_all_dic)
new_encrypt_fee_dic = get_new_encrypt_fee_dic(pd_fee)
new_encrypt_fee_dic # 这里再试一遍发现这样的方法推倒不出新的了

# 剩下的手动映射 然后更新
encrypt_fee_all_dic.update({"A1zWNNCi":0.5524,
"AlzWOdKm":1.5800,
"A1zRNtGm":0.2730,
"A1zbM9Wm":0.8270,
"A1zWNtem":0.5750,
"A1zaM9em":0.9250,
"A1zQMtOj":0.3315,
"A1zWNNqj":0.5585,
"A1zQMdGv":0.3039,
"A1zWMdGv":0.5039,
"A1zbNNSj":0.8565,
"AlzTNNSj":1.0565,
"A1zWM9Kn":0.5201,
"A1zbMtau":0.8348,
"A1zQMdCl":0.3023,
"AlzSOdan":1.1841,
"A1zRMNCv":0.2129,
"AlzRN9qu":1.2688,
"A1zQMtCu":0.3328,
"AlzTMtCu":1.0328,
"A1zXONCj":0.4925,
"A1zbMdWk":0.8072,
"A1zXN9av":0.4649,
"A1zUNtug":0.7796,
"A1zRNtGm":0.2730,
"A1zVM9Wm":0.6270,
"A1zQMtCu":0.3328,
"A1zUMtCu":0.7328,
"A1zSNNqv":0.1589,
"A1zQONKm":0.3900,
"A1zQMtOj":0.3315,
"A1zQNNqj":0.3585,
"A1zbNdCu":0.8428,
"A1zQMNqj":0.3185})

# 存起来
_time = datetime.datetime.now().__format__('%Y-%m-%d-%H-%M-%S')
b = json.dumps(encrypt_fee_all_dic)
f2 = open('encrypt_fee_all_dic%s.json' %(_time), 'w')
f2.write(b)
f2.close()

In [ ]:
# 使用映射完整解密价格详情表，与提供的价格表进行比较

In [159]:
pd_fee = pd.read_csv("./WanmaFeeEncrypt_v2.csv")

f = open("encrypt_fee_all_dic2021-04-27-14-22-40.json", "r")
content = f.read()
encrypt_fee_all_dic = json.loads(content)
f.close()
encrypt_fee_all_dic = dict(zip(encrypt_fee_all_dic, map(lambda x: round(x,4), encrypt_fee_all_dic.values()))) # 所有的值保留四位小数

for i in pd_fee.index:
    if pd_fee.loc[i, "electricityFee"] in encrypt_fee_all_dic.keys():
        pd_fee.loc[i, "electricityFee"] = encrypt_fee_all_dic[pd_fee.loc[i, "electricityFee"]]
    if pd_fee.loc[i, "serviceFee"] in encrypt_fee_all_dic.keys():
        pd_fee.loc[i, "serviceFee"] = encrypt_fee_all_dic[pd_fee.loc[i, "serviceFee"]]
    if pd_fee.loc[i, "totalFee"] in encrypt_fee_all_dic.keys():
        pd_fee.loc[i, "totalFee"] = encrypt_fee_all_dic[pd_fee.loc[i, "totalFee"]]

pd_fee_offered = pd.read_excel("./万马开放站点价格.xlsx")
pd_fee_offered.columns = ["province","cityname","name","feemodel","electricityFee","serviceFee","totalFee"]

pd_fee_type = pd_fee[["name","electricityFee","serviceFee","totalFee"]].drop_duplicates()
pd_fee_type_offered = pd_fee_offered[["name","electricityFee","serviceFee","totalFee"]].drop_duplicates()
pd_fee_type_offered = pd_fee_type_offered.round(4) # 保留四位小数 不然有点问题

In [423]:
pd_sameNumber_sameData = pd.DataFrame([], columns=["name", "electricityFee", "serviceFee", "totalFee", "name", "electricityFee", "serviceFee", "totalFee"])
pd_sameNumber_diffData = pd.DataFrame([], columns=["name", "electricityFee", "serviceFee", "totalFee", "name", "electricityFee", "serviceFee", "totalFee"])
pd_diffNumber_diffHigh = pd.DataFrame([], columns=["name", "electricityFee", "serviceFee", "totalFee", "name", "electricityFee", "serviceFee", "totalFee"])
pd_diffNumber_diffData = pd.DataFrame([], columns=["name", "electricityFee", "serviceFee", "totalFee", "name", "electricityFee", "serviceFee", "totalFee"])
pd_canNotFindInApp = pd.DataFrame([], columns=["name", "electricityFee", "serviceFee", "totalFee", "name", "electricityFee", "serviceFee", "totalFee"])

In [ ]:
def same_dataNumber(df1, df2):
    return len(df1) == len(df2)

for i in pd_fee_type_offered["name"].unique():
    df1 = pd_fee_type_offered[pd_fee_type_offered["name"]==i]
    df2 = pd_fee_type[pd_fee_type["name"]==i]
    
    df1 = df1.sort_values(["totalFee", "serviceFee"])
    df2 = df2.sort_values(["totalFee", "serviceFee"])
    df1.reset_index(inplace=True, drop=True)
    df2.reset_index(inplace=True, drop=True)
    
    if len(df2)==0: 
        pd_canNotFindInApp = pd_canNotFindInApp.append(pd.concat([df1, df2], axis=1)) # 表单里有但是app上搜不到

    elif same_dataNumber(df1, df2):
        samedata = (df1[["electricityFee", "serviceFee", "totalFee"]].values==df2[["electricityFee", "serviceFee", "totalFee"]].values).all()
        if samedata:
            pd_sameNumber_sameData = pd_sameNumber_sameData.append(pd.concat([df1, df2], axis=1)) # 行数相同，顺序对比，完全相同
        else:
            pd_sameNumber_diffData = pd_sameNumber_diffData.append(pd.concat([df1, df2], axis=1)) # 行数相同，顺序对比，有不同
    else:
        shortlen = min(len(df1), len(df2))
        samedata = (df1[:shortlen][["electricityFee", "serviceFee", "totalFee"]]==df2[:shortlen][["electricityFee", "serviceFee", "totalFee"]]).values.all()
        if samedata:
            pd_diffNumber_diffHigh = pd_diffNumber_diffHigh.append(pd.concat([df1, df2], axis=1)) # 行数不同，对比前半部分，相同
        else:
            pd_diffNumber_diffData = pd_diffNumber_diffData.append(pd.concat([df1, df2], axis=1)) # 行数不同，对比前半部分，不相同

In [ ]:
print(len(pd_diffNumber_diffHigh.iloc[:,0].unique()))
print(len(pd_sameNumber_sameData.iloc[:,0].unique()))

print(len(pd_diffNumber_diffData.iloc[:,0].unique()))
print(len(pd_sameNumber_diffData.iloc[:,0].unique()))

print(len(pd_canNotFindInApp.iloc[:,0].unique()))

pd_normal = pd_diffNumber_diffHigh.append(pd_sameNumber_sameData)
pd_unnormal = pd_sameNumber_diffData.append(pd_diffNumber_diffData)

_time = datetime.datetime.now().__format__('%Y-%m-%d-%H-%M-%S')
pd_normal.to_csv("./WanmaFeeCheck_normal%s.csv" %(_time), index = False, encoding="utf_8_sig")
pd_unnormal.to_csv("./WanmaFeeCheck_unnormal%s.csv" %(_time), index = False, encoding="utf_8_sig")
pd_canNotFindInApp.to_csv("./WanmaFeeCheck_canNotFindInApp%s.csv" %(_time), index = False, encoding="utf_8_sig")